# Tesla vs. GameStop Stock & Revenue Dashboard

In this notebook, we extract stock data using **yfinance** and revenue data via **web scraping** for Tesla (TSLA) and GameStop (GME). We then compare the two using interactive dashboards built with Plotly.

**Assignment Breakdown:**
- **Q1:** Extract Tesla Stock Data (yfinance) - 2 points
- **Q2:** Extract Tesla Revenue Data (Web Scraping) - 1 point
- **Q3:** Extract GameStop Stock Data (yfinance) - 2 points
- **Q4:** Extract GameStop Revenue Data (Web Scraping) - 1 point
- **Q5:** Tesla Stock vs Revenue Dashboard - 2 points
- **Q6:** GameStop Stock vs Revenue Dashboard - 2 points
- **Q7:** Share your assignment notebook (via GitHub) - 2 points

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

# For improved display of DataFrames
pd.set_option('display.max_columns', None)

## Question 1: Extracting Tesla Stock Data Using yfinance

In [2]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data (first 5 rows):")
print(tesla_data.head())

Tesla Stock Data (first 5 rows):
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


## Question 2: Extracting Tesla Revenue Data Using Web Scraping

Using a simple requests/BeautifulSoup approach, this cell attempts to extract Tesla's quarterly revenue data from Macrotrends. Note: If the table is not found because the page structure has changed, a message will be printed.

In [5]:


# Tesla revenue page URL on Macrotrends
url_tsla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Use headers to mimic a browser request
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/87.0.4280.66 Safari/537.36"
    )
}

# Make the HTTP GET request with headers
response = requests.get(url_tsla, headers=headers)

if response.status_code == 200:
    html_data_tsla = response.text
    soup_tsla = BeautifulSoup(html_data_tsla, "html.parser")

    tesla_revenue = None
    # Look through all tables for one that contains the phrase "Quarterly Revenue"
    for table in soup_tsla.find_all("table"):
        if "Quarterly Revenue" in table.get_text():
            try:
                tesla_revenue = pd.read_html(str(table))[0]
                break
            except Exception as e:
                print("Error parsing table:", e)

    if tesla_revenue is not None:
        if len(tesla_revenue.columns) >= 2:
            tesla_revenue.columns = ["Date", "Revenue"]
        else:
            print("Unexpected table structure.")

        tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
        tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r"\$", "", regex=True)\
                                                         .str.replace(",", "", regex=True)
        tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce")
        tesla_revenue.dropna(inplace=True)

        print("Tesla Revenue Data (first 5 rows):")
        print(tesla_revenue.head())

        print("Tesla Revenue Data (last 5 rows):")
        print(tesla_revenue.tail())
    else:
        print("Tesla revenue table not found.")
else:
    print("Failed to retrieve Tesla revenue page. Status code:", response.status_code)


Tesla Revenue Data (first 5 rows):
         Date  Revenue
0  2024-12-31  25707.0
1  2024-09-30  25182.0
2  2024-06-30  25500.0
3  2024-03-31  21301.0
4  2023-12-31  25167.0
Tesla Revenue Data (last 5 rows):
          Date  Revenue
57  2010-09-30     31.0
58  2010-06-30     28.0
59  2010-03-31     21.0
61  2009-09-30     46.0
62  2009-06-30     27.0


<ipython-input-5-9025d23ae3e6>:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tesla_revenue = pd.read_html(str(table))[0]


## Question 3: Extracting GameStop Stock Data Using yfinance

In [4]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("GameStop Stock Data (first 5 rows):")
print(gme_data.head())

GameStop Stock Data (first 5 rows):
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666417  1.666417  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


## Question 4: Extracting GameStop Revenue Data Using Web Scraping

In [9]:

# URL for GameStop revenue
url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# Define custom headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
}

# Make the GET request with headers
response_gme = requests.get(url_gme, headers=headers)

if response_gme.status_code == 200:
    html_data_gme = response_gme.text
    soup_gme = BeautifulSoup(html_data_gme, "html.parser")

    gme_revenue = None
    # Search through all tables for one that has "Quarterly Revenue" in its text
    for table in soup_gme.find_all("table"):
        if "Quarterly Revenue" in table.get_text():
            try:
                gme_revenue = pd.read_html(str(table))[0]
                break  # Stop once we find a matching table
            except Exception as e:
                print("Error parsing GameStop table:", e)

    if gme_revenue is not None:
        # Expecting at least two columns (Date and Revenue)
        if len(gme_revenue.columns) >= 2:
            gme_revenue.columns = ["Date", "Revenue"]
        else:
            print("Unexpected GameStop table structure.")

        # Clean the revenue data
        gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
        gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r"\$", "", regex=True).str.replace(",", "", regex=True)
        gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors="coerce")
        gme_revenue.dropna(inplace=True)

        # Display the resulting DataFrame
        print("GameStop Revenue Data (first 5 rows):")
        print(gme_revenue.head())
        print("\nGameStop Revenue Data (last 5 rows):")
        print(gme_revenue.tail())
    else:
        print("GameStop revenue table not found.")
else:
    print("Failed to retrieve GameStop revenue page. Status code:", response_gme.status_code)

GameStop Revenue Data (first 5 rows):
         Date  Revenue
0  2025-01-31     1283
1  2024-10-31      860
2  2024-07-31      798
3  2024-04-30      882
4  2024-01-31     1794

GameStop Revenue Data (last 5 rows):
          Date  Revenue
60  2010-01-31     3524
61  2009-10-31     1835
62  2009-07-31     1739
63  2009-04-30     1981
64  2009-01-31     3492


<ipython-input-9-3a986bfe228d>:23: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



## Question 5: Tesla Stock and Revenue Dashboard

In [7]:
# Create dashboard for Tesla (stock vs revenue)
fig = go.Figure()
fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Close'], name="Tesla Stock Price"))
if 'tesla_revenue' in globals() and tesla_revenue is not None:
    try:
        tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'], errors='coerce')
    except Exception as e:
        print("Error converting Tesla revenue dates:", e)
    fig.add_trace(go.Scatter(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'], name="Tesla Revenue"))
else:
    print("Tesla revenue data is unavailable.")

fig.update_layout(title="Tesla Stock Price vs Revenue",
                  xaxis_title="Date",
                  yaxis_title="USD")
fig.show()

## Question 6: GameStop Stock and Revenue Dashboard

In [8]:
# Create dashboard for GameStop (stock vs revenue)
fig = go.Figure()
fig.add_trace(go.Scatter(x=gme_data['Date'], y=gme_data['Close'], name="GameStop Stock Price"))
if 'gme_revenue' in globals() and gme_revenue is not None:
    try:
        gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'], errors='coerce')
    except Exception as e:
        print("Error converting GameStop revenue dates:", e)
    fig.add_trace(go.Scatter(x=gme_revenue['Date'], y=gme_revenue['Revenue'], name="GameStop Revenue"))
else:
    print("GameStop revenue data is unavailable.")

fig.update_layout(title="GameStop Stock Price vs Revenue",
                  xaxis_title="Date",
                  yaxis_title="USD")
fig.show()

GameStop revenue data is unavailable.


## Question 7: Sharing Your Assignment Notebook

Upload your completed notebook to a public GitHub repository and share the URL as required by the assignment.